<a href="https://colab.research.google.com/github/msyashik/CPPS/blob/master/Copy_of_GAN_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline
import tensorflow as tf

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.layers import Dense, Reshape
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model

block_size = 128
num_blocks=4
OPTIMIZER = tf.keras.optimizers.Adam(lr=0.0002, decay=8e-9)

generator = Sequential()
generator.add(Dense(block_size, input_shape=(100, )))
generator.add(LeakyReLU(alpha=0.02))
generator.add(BatchNormalization(momentum=0.8))

for i in range(num_blocks-1):
    block_size = block_size * 2
    generator.add(Dense(block_size))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(28 * 28 * 1, activation='tanh'))
generator.add(Reshape((28, 28, 1)))
generator.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [5]:
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               12928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 128)               512       
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
dense_7 (Dense)              (None, 512)              

In [6]:
import sys
import numpy as np
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model

SHAPE =  (28, 28, 1)
CAPACITY = 28 * 28 * 1
OPTMIZER = tf.keras.optimizers.Adam(lr=0.0002, decay=8e-9)

discriminator = Sequential()
discriminator.add(Flatten(input_shape = SHAPE))
discriminator.add(Dense(CAPACITY, input_shape=SHAPE))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(int(CAPACITY/2)))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
discriminator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 784)               615440    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 784)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 392)               307720    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 392)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 393       
Total params: 923,553
Trainable params: 923,553
Non-trainable params: 0
________________________________________________

In [8]:
OPTIMIZER = tf.keras.optimizers.Adam(lr=0.0002, decay=8e-9)
discriminator.trainable = False

gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:
gan.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 28, 28, 1)         1514128   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 923553    
Total params: 2,437,681
Trainable params: 1,510,288
Non-trainable params: 927,393
_________________________________________________________________


In [10]:
def load_MNIST(model_type=-1):
    allowed_types = [-1,0,1,2,3,4,5,6,7,8,9]
    if model_type not in allowed_types:
        print('ERROR: Only Integer Values from -1 to 9 are allowed')

    (X_train, Y_train), (_, _) = mnist.load_data()
    if model_type!=-1:
        X_train = X_train[np.where(Y_train==int(model_type))[0]]
    
    # Rescale -1 to 1
    # Find Normalize Function from CV Class  
    X_train = ( np.float32(X_train) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    return X_train

In [11]:
W = 28
H = 28
C = 1
EPOCHS = 5000001
BATCH = 32
CHECKPOINT = 500
model_type=-1
LATENT_SPACE_SIZE = 100

# generator = Generator(height=H, width=W, channels=C, latent_size=LATENT_SPACE_SIZE)
# discriminator = Discriminator(height=H, width=W, channels=C)
# gan = GAN(generator=generator.Generator, discriminator=discriminator.Discriminator)

X_train = load_MNIST()



def train():
    

    for e in range(EPOCHS):
        # Train Discriminator
        # Make the training batch for this model be half real, half noise
        # Grab Real Images for this training batch
        count_real_images = int(BATCH/2)
        starting_index = randint(0, (len(X_train)-count_real_images))
        real_images_raw = X_train[ starting_index : (starting_index + count_real_images) ]
        x_real_images = real_images_raw.reshape( count_real_images, W, H, C )
        y_real_labels = np.ones([count_real_images,1])

        # Grab Generated Images for this training batch
        latent_space_samples = sample_latent_space(count_real_images)
        x_generated_images = generator.predict(latent_space_samples)
        y_generated_labels = np.zeros([BATCH-count_real_images,1])

        # Combine to train on the discriminator
        x_batch = np.concatenate( [x_real_images, x_generated_images] )
        y_batch = np.concatenate( [y_real_labels, y_generated_labels] )

        # Now, train the discriminator with this batch
        discriminator_loss = discriminator.train_on_batch(x_batch,y_batch)

        # Generate Noise
        x_latent_space_samples = sample_latent_space(BATCH)
        y_generated_labels = np.ones([BATCH,1])
        generator_loss = gan.train_on_batch(x_latent_space_samples,y_generated_labels)

        

        if e % CHECKPOINT == 0 :
            print ('Epoch: '+str(int(e))+', [Discriminator :: Loss: '+str(discriminator_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
            plot_checkpoint(e)
    return

11501568/11490434 [==============================] - 0s 0us/step


In [12]:
def sample_latent_space(instances):
    return np.random.normal(0, 1, (instances, LATENT_SPACE_SIZE))

def plot_checkpoint(e):
    filename = "/content/sample_"+str(e)+".png"
    gfilename = "/content/gdrive/MyDrive/Colab Notebooks/Deep Learning/GAN/MNIST samples/ample_"+str(e)+".png"

    noise = sample_latent_space(16)
    images = generator.predict(noise)
    
    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [H,W])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename)
    plt.savefig(gfilename)
    plt.close('all')
    return

In [ ]:
train()

Epoch: 0, [Discriminator :: Loss: 0.07501557469367981], [ Generator :: Loss: 4.3932600021362305]
Epoch: 500, [Discriminator :: Loss: 0.10127374529838562], [ Generator :: Loss: 5.102730751037598]
Epoch: 1000, [Discriminator :: Loss: 0.024722114205360413], [ Generator :: Loss: 5.502975940704346]
Epoch: 1500, [Discriminator :: Loss: 0.4637097716331482], [ Generator :: Loss: 4.1884331703186035]
Epoch: 2000, [Discriminator :: Loss: 0.367364764213562], [ Generator :: Loss: 3.777400255203247]


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(plt.imread('/content/Data/smple_639500.png'))

FileNotFoundError: ignored